In [3]:
%matplotlib inline

import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

data_path= '..'

from indirect import run_model
from natsort import natsorted

In [4]:
EPSILON = sys.float_info.epsilon  # smallest possible difference

def get_recon_curve(ini_curve,timesteps):
    if timesteps == 0:
        return [0,0]

    def interpolate(inp, fi):
        i = int(fi)
        f = fi - i
        return (inp[i] if f < EPSILON else
                inp[i] + f*(inp[i+1]-inp[i]))

    inp = ini_curve
    new_len = timesteps

    delta = (len(inp)-1) / float(new_len-1)
    outp = np.diff([1-int(interpolate(inp, i*delta))/100 for i in range(new_len)])
    #outp = [1-int(interpolate(inp, i*delta))/100 for i in range(new_len)]
    return list(outp)

### Load IO Data

In [6]:
IO_TABLE = pd.read_csv(os.path.join(data_path,'data','Rijnmond_IO.csv'),index_col=[0],header=[0])
SECTORS = list(IO_TABLE[:25].index.get_level_values(0).unique())

### Load direct impacts

### Run economic impact model

In [ ]:
%%time
scenarios = list(direct_impacts.index.get_level_values(0).unique())
diff_recons_tot = {}
diff_recons_ind = {}
curve_types = ['linear','convex','concave']
concave = np.array([100,98,93,88,83,75,64,53,38,20,0])
linear = np.array([100,90,80,70,60,50,40,30,20,10,0])
convex = np.array([100,70,55,40,30,20,15,10,5,0,0])

for recon_type,curve_type in zip([linear,convex,concave],curve_types): #
       
    get_losses_tot = {}
    get_losses_ind = {}

    for scen, new_df in tqdm(direct_impacts.groupby(level=0),total=len(direct_impacts.groupby(level=0))):
        # create reconstruction matrix
        all_sectors = [get_recon_curve(recon_type,int(x)) for x in list(JustFlood[(int(scen.split('_')[0][1:]),int(scen.split('_')[1]))])]
        pad = max(len(max(all_sectors, key=len))+1,180)
        recon_matrix = np.array([i + [0]*(pad-len(i)) for i in all_sectors])

        # perform calculations
        new_df = new_df.copy()
        new_df.loc[new_df.rel_m2_inun > 1] = 0.99
        ratio_df = new_df.merge(corr_df.reset_index(),left_on='landuse',right_on='index')
        rel_impact = dict(zip(ratio_df.landuse,ratio_df.rel_m2_inun*ratio_df.ratio))
        
        get_losses_tot[scen] = (run_model([rel_impact[x] if x in rel_impact.keys() else 0 for x in SECTORS],recon_matrix,3,pad,dict(direct_damages[scen])))[0]
        get_losses_ind[scen] = (run_model([rel_impact[x] if x in rel_impact.keys() else 0 for x in SECTORS],recon_matrix,3,pad,dict(direct_damages[scen])))[1]

    df_tot = pd.DataFrame.from_dict(get_losses_tot,orient='index')
    df_tot.index = pd.MultiIndex.from_tuples([x.split('_')[0:2] for x in scenarios])
    df_tot = df_tot.unstack(0)
    df_tot.index = df_tot.index.astype(int)
    df_tot = df_tot.sort_index()
    df_tot.columns = df_tot.columns.droplevel(0)
    diff_recons_tot[curve_type] = df_tot

    df_ind =  pd.DataFrame.from_dict(get_losses_ind,orient='index',columns=SECTORS)
    df_ind.index = pd.MultiIndex.from_tuples([x.split('_')[0:2] for x in scenarios])
    df_ind = df_ind.unstack(0)
    df_ind.index = df_ind.index.astype(int)
    df_ind = df_ind.sort_index()
    diff_recons_ind[curve_type] = df_ind